In [1]:
import re
import requests
import pandas as pd
import numpy as np

In [2]:
#### Importing csv data files into dataframes 
df = pd.read_csv('education_expenditure_supplementary_data.csv')#Public and private direct expenditures on education institutions as a percentage of gross domestic product
df1 = pd.read_csv('educational_attainment_supplementary_data.csv')#refined list of survey questions by country 
df2 = pd.read_csv('school_and_country_table.csv')
df3 = pd.read_csv('shanghaiData.csv')
df4 = pd.read_csv('timesData.csv')
df_GDP = pd.read_csv('GDP.csv')
df_GDP2 = pd.read_csv('GDP.csv')

In [3]:
#Cleaning GDP file and averaging out years 2001 through 2011. This will help smooth out missing data and lag between school investments vs results 
df_GDP.rename({'Country Name': 'country'}, axis=1, inplace=True)
df_GDP['avg_2000_2011'] = df_GDP[['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009', '2010','2011']].mean(axis=1)
df_GDP = df_GDP[['country', 'Country Code','Indicator Name', 'Indicator Code','avg_2000_2011']]
df_GDP.dropna(subset=['avg_2000_2011'],inplace=True)
df_GDP.reset_index(inplace=True,drop=True)
df_GDP.head()


,country,Country Code,Indicator Name,Indicator Code,avg_2000_2011
0,World,WLD,GDP (current US$),NY.GDP.MKTP.CD,5.373832e+13
1,High income,HIC,GDP (current US$),NY.GDP.MKTP.CD,3.805390e+13
2,OECD members,OED,GDP (current US$),NY.GDP.MKTP.CD,3.787817e+13
3,Post-demographic dividend,PST,GDP (current US$),NY.GDP.MKTP.CD,3.594110e+13
4,IDA & IBRD total,IBT,GDP (current US$),NY.GDP.MKTP.CD,1.618787e+13


In [4]:
#Cleaning school expenditure file. This file is missing many countries (only has around 30)
#Filtering on school spend and creating dataframe for each by category - e.g. All, k_12, and hihger education
#We may be able to run test on all 3 of these groups 
df['institute_type'] = df['institute_type'].str.strip()
df['direct_expenditure_type'] = df['direct_expenditure_type'].str.strip()

df[['1995','2000','2005','2009','2010','2011']] = df[['1995','2000','2005','2009','2010','2011']]/100
df['school_spend_2000_2011'] = df[['2000','2005','2009', '2010','2011']].mean(axis=1)

df = df[df['direct_expenditure_type']== 'Total']
df = df[['country', 'institute_type','school_spend_2000_2011']]

df_all_edu = df[df['institute_type']=='All Institutions']
df_k_12_edu = df[df['institute_type']=='Elementary and Secondary Institutions']
df_higher_edu = df[df['institute_type']=='Higher Education Institutions']

df_all_edu = df_all_edu.dropna(subset=['school_spend_2000_2011'])
df_k_12_edu = df_k_12_edu.dropna(subset=['school_spend_2000_2011'])
df_higher_edu = df_higher_edu.dropna(subset=['school_spend_2000_2011'])

df_all_edu.reset_index(inplace=True,drop=True)
df_k_12_edu.reset_index(inplace=True,drop=True)
df_higher_edu.reset_index(inplace=True,drop=True)

df_all_edu.head()
df_k_12_edu.head()
df_higher_edu.head()

#df.institute_type.unique()

,country,institute_type,school_spend_2000_2011
0,OECD Average,Higher Education Institutions,0.016
1,Australia,Higher Education Institutions,0.016
2,Austria,Higher Education Institutions,0.015
3,Belgium,Higher Education Institutions,0.014
4,Chile,Higher Education Institutions,0.024


In [5]:
#Merging GDP and expenditure data. The column "school expenditures will be the $$$ spend on schooling"
df_all_edu_1 = df_GDP.merge(df_all_edu, on='country', how='left').dropna(subset=['school_spend_2000_2011'])
df_k_12_edu_1 = df_GDP.merge(df_k_12_edu, on='country', how='left').dropna(subset=['school_spend_2000_2011'])
df_higher_edu_1 = df_GDP.merge(df_higher_edu, on='country', how='left').dropna(subset=['school_spend_2000_2011'])

df_all_edu_1.reset_index(inplace=True,drop=True)
df_k_12_edu_1.reset_index(inplace=True,drop=True)
df_higher_edu_1.reset_index(inplace=True,drop=True)

df_all_edu_1['school_expenditures'] = round(df_all_edu_1['school_spend_2000_2011'] * df_all_edu_1['avg_2000_2011'],2)
df_k_12_edu_1['school_expenditures'] = round(df_k_12_edu_1['school_spend_2000_2011'] * df_k_12_edu_1['avg_2000_2011'],2)
df_higher_edu_1['school_expenditures'] = round(df_higher_edu_1['school_spend_2000_2011'] * df_higher_edu_1['avg_2000_2011'],2)

df_all_edu_1.head()
df_k_12_edu_1.head()
df_higher_edu_1.head()


,country,Country Code,Indicator Name,Indicator Code,avg_2000_2011,institute_type,school_spend_2000_2011,school_expenditures
0,United States,USA,GDP (current US$),NY.GDP.MKTP.CD,1.303684e+13,Higher Education Institutions,0.027,3.519947e+11
1,Japan,JPN,GDP (current US$),NY.GDP.MKTP.CD,4.944989e+12,Higher Education Institutions,0.016,7.911982e+10
2,Germany,DEU,GDP (current US$),NY.GDP.MKTP.CD,2.899084e+12,Higher Education Institutions,0.013,3.768809e+10
3,United Kingdom,GBR,GDP (current US$),NY.GDP.MKTP.CD,2.372315e+12,Higher Education Institutions,0.012,2.846777e+10
4,France,FRA,GDP (current US$),NY.GDP.MKTP.CD,2.206148e+12,Higher Education Institutions,0.015,3.309223e+10


In [6]:
### data cleaning the survey file to narrow in on questions we may want to take a look at. Column "series name"
df1 = df1[df1['Keep or not']==1]
df1['avg_2005_2015'] = df1[['2005', '2006','2007','2008','2009','2010','2011','2012','2013','2015']].mean(axis=1)
df1.dropna(subset=['avg_2005_2015'],inplace=True)
df1.reset_index(inplace=True,drop=True)
df1.head()

,country_name,series_name,Keep or not,1985,1986,1987,1990,1991,1992,1993,...,2006,2007,2008,2009,2010,2011,2012,2013,2015,avg_2005_2015
0,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,0.50,NaN,NaN,0.66,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.97,NaN,NaN,NaN,NaN,1.620
1,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,1.57,NaN,NaN,1.87,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.85,NaN,NaN,NaN,NaN,3.530
2,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,0.28,NaN,NaN,0.42,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.16,NaN,NaN,NaN,NaN,1.080
3,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,1.16,NaN,NaN,1.49,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.23,NaN,NaN,NaN,NaN,2.915
4,Afghanistan,Barro-Lee: Percentage of population age 15+ wi...,1.0,77.80,NaN,NaN,75.04,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,58.38,NaN,NaN,NaN,NaN,62.055


In [7]:
#cleaning GDP file to use again. As many countries are missing expenditures, we can just use toal GDP vs school responses as well. This will be a broader dataset with more countries.
df_GDP2.rename({'Country Name': 'country'}, axis=1, inplace=True)
df_GDP2['avg_2000_2015'] = df_GDP2[['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009', '2010','2011','2012','2013','2014','2015']].mean(axis=1)
df_GDP2['avg_2000_2015'] = df_GDP2['avg_2000_2015']/1000000000
df_GDP2 = df_GDP2[['country', 'Country Code','Indicator Name', 'Indicator Code','avg_2000_2015']]
df_GDP2.dropna(subset=['avg_2000_2015'],inplace=True)
df_GDP2.reset_index(inplace=True,drop=True)
df_GDP2.head()

,country,Country Code,Indicator Name,Indicator Code,avg_2000_2015
0,World,WLD,GDP (current US$),NY.GDP.MKTP.CD,59517.199213
1,High income,HIC,GDP (current US$),NY.GDP.MKTP.CD,40900.271146
2,OECD members,OED,GDP (current US$),NY.GDP.MKTP.CD,40629.787880
3,Post-demographic dividend,PST,GDP (current US$),NY.GDP.MKTP.CD,38437.087014
4,IDA & IBRD total,IBT,GDP (current US$),NY.GDP.MKTP.CD,19195.578598


In [8]:
#import country_cleaner.country_clean as country_clean
#df_GDP2["Country Name"] = df_GDP2["Country Name"].apply(lambda x: country_clean(x))

In [9]:
# joinning survey data_sets with expenditures dataset for final data sets to beging running test on.
df1.rename({'country_name': 'country'}, axis=1, inplace=True)

complete_expenditures_all = df1.merge(df_all_edu_1, on='country', how='left').dropna(subset=['school_spend_2000_2011'])
complete_expenditures_k_12 = df1.merge(df_k_12_edu_1, on='country', how='left').dropna(subset=['school_spend_2000_2011'])
complete_expenditures_higher_edu = df1.merge(df_higher_edu_1, on='country', how='left').dropna(subset=['school_spend_2000_2011'])


In [10]:
#final data-set with total GDP (not expenditures) to run test on as it will have more datapoints vs above compplete tables
complete_GDP = df1.merge(df_GDP2, on='country', how='left').dropna(subset=['avg_2000_2015'])

In [11]:
complete_GDP.head(5)

,country,series_name,Keep or not,1985,1986,1987,1990,1991,1992,1993,...,2010,2011,2012,2013,2015,avg_2005_2015,Country Code,Indicator Name,Indicator Code,avg_2000_2015
0,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,0.50,NaN,NaN,0.66,NaN,NaN,NaN,...,1.97,NaN,NaN,NaN,NaN,1.620,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
1,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,1.57,NaN,NaN,1.87,NaN,NaN,NaN,...,3.85,NaN,NaN,NaN,NaN,3.530,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
2,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,0.28,NaN,NaN,0.42,NaN,NaN,NaN,...,1.16,NaN,NaN,NaN,NaN,1.080,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
3,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,1.16,NaN,NaN,1.49,NaN,NaN,NaN,...,3.23,NaN,NaN,NaN,NaN,2.915,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
4,Afghanistan,Barro-Lee: Percentage of population age 15+ wi...,1.0,77.80,NaN,NaN,75.04,NaN,NaN,NaN,...,58.38,NaN,NaN,NaN,NaN,62.055,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438


In [12]:
complete_GDP = complete_GDP[(complete_GDP['series_name']=='Barro-Lee: Average years of total schooling, age 25+, total') |(complete_GDP['series_name']=='Barro-Lee: Average years of total schooling, age 25+, female') | (complete_GDP['series_name']=='Barro-Lee: Percentage of population age 25+ with no education')]
complete_GDP.head()                         

,country,series_name,Keep or not,1985,1986,1987,1990,1991,1992,1993,...,2010,2011,2012,2013,2015,avg_2005_2015,Country Code,Indicator Name,Indicator Code,avg_2000_2015
2,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,0.28,NaN,NaN,0.42,NaN,NaN,NaN,...,1.16,NaN,NaN,NaN,NaN,1.080,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
3,Afghanistan,"Barro-Lee: Average years of total schooling, a...",1.0,1.16,NaN,NaN,1.49,NaN,NaN,NaN,...,3.23,NaN,NaN,NaN,NaN,2.915,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
5,Afghanistan,Barro-Lee: Percentage of population age 25+ wi...,1.0,84.62,NaN,NaN,80.70,NaN,NaN,NaN,...,66.02,NaN,NaN,NaN,NaN,68.800,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
8,Albania,"Barro-Lee: Average years of total schooling, a...",1.0,6.13,NaN,NaN,7.29,NaN,NaN,NaN,...,9.57,NaN,NaN,NaN,NaN,9.470,ALB,GDP (current US$),NY.GDP.MKTP.CD,9.476655
9,Albania,"Barro-Lee: Average years of total schooling, a...",1.0,6.75,NaN,NaN,7.83,NaN,NaN,NaN,...,9.85,NaN,NaN,NaN,NaN,9.770,ALB,GDP (current US$),NY.GDP.MKTP.CD,9.476655


In [13]:
df_pivot=complete_GDP.pivot(index=['country'], columns='series_name',values=['avg_2005_2015'])
df_pivot.columns = df_pivot.columns.droplevel()
modified_df=df_pivot.rename_axis(None,axis=1)
modified_df=modified_df.reset_index()
modified_df_final = modified_df.merge(df_GDP2, on='country', how='left').dropna(subset=['avg_2000_2015'])
modified_df_final

,country,"Barro-Lee: Average years of total schooling, age 25+, female","Barro-Lee: Average years of total schooling, age 25+, total",Barro-Lee: Percentage of population age 25+ with no education,Country Code,Indicator Name,Indicator Code,avg_2000_2015
0,Afghanistan,1.080,2.915,68.800,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
1,Albania,9.470,9.770,3.185,ALB,GDP (current US$),NY.GDP.MKTP.CD,9.476655
2,Algeria,4.835,5.570,31.295,DZA,GDP (current US$),NY.GDP.MKTP.CD,133.920615
3,Argentina,9.345,9.370,1.100,ARG,GDP (current US$),NY.GDP.MKTP.CD,345.568498
4,Armenia,10.795,10.835,0.490,ARM,GDP (current US$),NY.GDP.MKTP.CD,7.306080
...,...,...,...,...,...,...,...,...
138,"Venezuela, RB",7.850,7.655,8.460,VEN,GDP (current US$),NY.GDP.MKTP.CD,245.228688
139,Vietnam,6.500,6.945,7.830,VNM,GDP (current US$),NY.GDP.MKTP.CD,96.776592
140,"Yemen, Rep.",1.065,2.230,73.060,YEM,GDP (current US$),NY.GDP.MKTP.CD,24.405175
141,Zambia,5.605,6.460,19.755,ZMB,GDP (current US$),NY.GDP.MKTP.CD,14.815902


In [14]:
modified_df_final.rename({'Barro-Lee: Average years of total schooling, age 25+, female': 'female_25_plus','Barro-Lee: Average years of total schooling, age 25+, total': 'All_25_plus','Barro-Lee: Percentage of population age 25+ with no education': 'All_25_plus_no_edu'}, axis=1, inplace=True)
modified_df_final.head()

,country,female_25_plus,All_25_plus,All_25_plus_no_edu,Country Code,Indicator Name,Indicator Code,avg_2000_2015
0,Afghanistan,1.080,2.915,68.800,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438
1,Albania,9.470,9.770,3.185,ALB,GDP (current US$),NY.GDP.MKTP.CD,9.476655
2,Algeria,4.835,5.570,31.295,DZA,GDP (current US$),NY.GDP.MKTP.CD,133.920615
3,Argentina,9.345,9.370,1.100,ARG,GDP (current US$),NY.GDP.MKTP.CD,345.568498
4,Armenia,10.795,10.835,0.490,ARM,GDP (current US$),NY.GDP.MKTP.CD,7.306080


In [15]:
modified_df_final['country_id'] = pd.factorize(modified_df_final.country)[0]

In [16]:
modified_df_final.head()

,country,female_25_plus,All_25_plus,All_25_plus_no_edu,Country Code,Indicator Name,Indicator Code,avg_2000_2015,country_id
0,Afghanistan,1.080,2.915,68.800,AFG,GDP (current US$),NY.GDP.MKTP.CD,12.328438,0
1,Albania,9.470,9.770,3.185,ALB,GDP (current US$),NY.GDP.MKTP.CD,9.476655,1
2,Algeria,4.835,5.570,31.295,DZA,GDP (current US$),NY.GDP.MKTP.CD,133.920615,2
3,Argentina,9.345,9.370,1.100,ARG,GDP (current US$),NY.GDP.MKTP.CD,345.568498,3
4,Armenia,10.795,10.835,0.490,ARM,GDP (current US$),NY.GDP.MKTP.CD,7.306080,4


In [17]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer


X_complete_GDP = modified_df_final[['female_25_plus','All_25_plus','All_25_plus_no_edu','avg_2000_2015']]
y_complete_GDP = modified_df_final[['country_id']]
X_complete_GDP

,female_25_plus,All_25_plus,All_25_plus_no_edu,avg_2000_2015
0,1.080,2.915,68.800,12.328438
1,9.470,9.770,3.185,9.476655
2,4.835,5.570,31.295,133.920615
3,9.345,9.370,1.100,345.568498
4,10.795,10.835,0.490,7.306080
...,...,...,...,...
138,7.850,7.655,8.460,245.228688
139,6.500,6.945,7.830,96.776592
140,1.065,2.230,73.060,24.405175
141,5.605,6.460,19.755,14.815902


In [18]:
X_complete_GDP_normalized = MinMaxScaler().fit(X_complete_GDP).transform(X_complete_GDP)  
kmeans = KMeans(n_clusters = 5, random_state = 0)
kmeans.fit(X_complete_GDP)
kmeans.labels_
modified_df_final['cluster'] = kmeans.fit_predict(X_complete_GDP)
y_kmeans = kmeans.predict(X_complete_GDP)
kmeans.labels_
kmeans.cluster_centers_


array([[7.20852000e+00, 7.58896000e+00, 2.02831600e+01, 9.02871700e+01],
       [1.32750000e+01, 1.32750000e+01, 4.45000000e-01, 1.40743376e+04],
       [9.26300000e+00, 9.48800000e+00, 7.36900000e+00, 2.42373195e+03],
       [9.09200000e+00, 9.54350000e+00, 9.33150000e+00, 1.07619918e+03],
       [8.78000000e+00, 9.16750000e+00, 4.10000000e+00, 4.98019991e+03]])